In [ ]:
!pip install streamlit
!pip install openai
!pip install pyngrok

In [ ]:
!ngrok authtoken

현재 날씨 알려주는 챗봇 만들기

https://openweathermap.org/api

In [ ]:
# weathermap_api_key
weather_api_key= "_________________"

In [ ]:
import requests

def get_current_weather(location, format):
  # OpenWeatherMap API 키를 'your_api_key_here' 자리에 실제 키로 교체하세요
  api_key = 'your_api_key_here'
  # 현재 날씨 정보를 위한 OpenWeatherMap API 엔드포인트
  base_url = "http://api.openweathermap.org/data/2.5/weather?"

  # API 요청을 위한 온도 단위 시스템을 포맷에 따라 변환
  units = 'metric' if format == 'celsius' else 'imperial'

  # 파라미터를 포함하여 완성된 API URL 구성
  complete_url = f"{base_url}appid={api_key}&q={location}&units={units}"

  # API 요청 수행
  response = requests.get(complete_url)
  print(response)
  # 요청이 성공적이었는지 확인
  if response.status_code == 200:
    # 응답 JSON 파싱
    data = response.json()

    # 관련 날씨 정보 추출
    temperature = data['main']['temp']
    weather_description = data['weather'][0]['description']

    # 결과 문자열 구성
    result = (f"'location':{location}, 'temperature':{temperature},'format':{format}, 'weather_description':{weather_description}")
    return result
  else:
    # 가능한 오류 처리(예: 위치를 찾을 수 없음)
    return "날씨 정보를 가져올 수 없습니다. 위치를 확인하고 다시 시도하세요."

In [ ]:
print(get_current_weather(location ="seoul",format="celsius"))

In [ ]:
tools = [
  {
    "type": "function",
    "function": {
      "name": "get_current_weather",
      "description": "현재 위치의 현재 날씨를 확인합니다.",
      "parameters": {
        "type": "object",
        "properties": {
          "location": {
            "type": "string",
            "description": "도시(예: 서울)",
          },
          "format": {
            "type": "string",
            "enum": ["celsius", "fahrenheit"],
            "description": "사용할 온도 단위입니다. 사용자 위치에서 이를 유추합니다.",
          },
        },
        "required": ["location", "format"],
      },
    }
  }]

In [ ]:
%%writefile app.py

import streamlit as st
from openai import OpenAI
import json
import requests

API_KEY = ""
weather_api_key= ""

st.title("날씨봇🌤️")

# Set OpenAI API key from Streamlit secrets
client = OpenAI(api_key=API_KEY)

# Set a default model
if "openai_model" not in st.session_state:
    st.session_state.openai_model = "gpt-3.5-turbo"

# Initialize chat history
if "messages" not in st.session_state:
    st.session_state.messages = [{
      "role": "system",
      "content": "함수에 연결할 값을 가정하지 마세요.\
      사용자의 요청이 모호한 경우 설명을 요청하세요\
      한국어로 친절하게 이모지를 섞어서 답변해줘"
      }]


def get_current_weather(location, format):
    # Replace 'your_api_key_here' with your actual OpenWeatherMap API key
    api_key = weather_api_key
    # OpenWeatherMap API endpoint for current weather
    base_url = "http://api.openweathermap.org/data/2.5/weather?"

    # Convert the format to the corresponding unit system for the API request
    units = 'metric' if format == 'celsius' else 'imperial'

    # Construct the complete API URL with parameters
    complete_url = f"{base_url}appid={api_key}&q={location}&units={units}"

    # Perform the API request
    response = requests.get(complete_url)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the response JSON
        data = response.json()

        # Extract relevant weather information
        temperature = data['main']['temp']
        weather_description = data['weather'][0]['description']

        # Construct a result string
        result = (f"'location:{location}, 'temperature':{temperature},'format':{format}, 'weather_description':{weather_description}")
        return result
    else:
        # Handle possible errors (e.g., location not found)
        return "Weather information could not be retrieved. Please check the location and try again."

tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. Seoul",
                    },
                    "format": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "The temperature unit to use. Infer this from the users location.",
                    },
                },
                "required": ["location", "format"],
            },
        }
    }]


# Display chat messages from history on app rerun
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

# Accept user input for chat
if prompt := st.chat_input("날씨에 대해 물어보세요"):
    # Add user message to chat history
    st.session_state.messages.append({"role": "user", "content": prompt})

    # Display user message in chat message container
    with st.chat_message("user"):
        st.markdown(prompt)

    # Display assistant response in chat message container
    with st.chat_message("assistant"):
        message_placeholder = st.empty()
        full_response = ""
        response = client.chat.completions.create(
            model=st.session_state.openai_model,
            messages=st.session_state.messages,
            tools=tools,
            )
        response_message = response.choices[0].message
        function_name = response_message.tool_calls[0].function.name
        arguments = json.loads(response_message.tool_calls[0].function.arguments)
# 함수에 인자값을 넣어 실행하기
        function_response = eval(f"{function_name}(**arguments)")

        st.session_state.messages.append({
          "role":"function",
          "name":function_name,
          "content":function_response
        })

        # 여기부터 시작해야 댈듯
        for response in client.chat.completions.create(
            model=st.session_state.openai_model,
            messages=st.session_state.messages,
            tools = tools,
            stream=True,
        ):
            full_response += (response.choices[0].delta.content or "")
            message_placeholder.markdown(full_response + "▌")
        message_placeholder.markdown(full_response)
    st.session_state.messages.append({"role": "assistant", "content": full_response})


In [ ]:
!ps

In [ ]:
!kill <streamlit 번호>

In [ ]:
ngrok.kill()